# Location

The `location` directive, used within the `server` context, specifies how NGINX should process requests based on the URI.

More in [correspoding section](http://nginx.org/en/docs/http/ngx_http_core_module.html#location) of the official documentation.

## Setup

It turns out that it's quite a complex task to build examples that show how everything works, so this section describes what we need to show everything. In summary, we need

- Proxied server - server where we'll redirect requests to nginx.
- And nginx, which can be configured differently for different examples.
- A network that connects containers.

The following cell creates a Docker Compose file that satisfies all these requirements.

In [20]:
cat << EOF > location_files/docker-compose.yml
services:
  proxied:
    image: kennethreitz/httpbin
    container_name: client_container
    ports: 
      - 81:80
  nginx:
    image: nginx
    container_name: experiment_nginx
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf
    ports: 
      - 80:80
EOF

The following cell runs applications.

In [21]:
docker compose -f location_files/docker-compose.yml up -d &> /dev/null

**Note** Don't forget to clear the environment.

In [15]:
docker compose -f location_files/docker-compose.yml down &> /dev/null

## URI matching

The `location` directive has an argument that specifies rules for which URI to match in which context. This section considers several options.

---

In the following example, nginx config defines two locations:  `/pattern` and `/pattern/` URIs. By looking at the output, you can see exactly which location was triggered. They're almost the same, but you'll see that they match the different cases.

In [22]:
cat << EOF > location_files/nginx.conf
events {}
http {
    server {
        listen 80;
        
        location /pattern {
            return 200 "location 1 - I use /pattern";
        }

        location /pattern/ {
            return 200 "location 2 - I use /pattern/";
        }
    }
}
EOF

docker exec -it experiment_nginx nginx -s reload

2024/09/05 09:13:54 [notice] 30#30: signal process started


Now we can request different URIs from nginx and see what exactly has been triggered.

The following example shows what happens when you access `/pattern` (no `/` at the end of the command).

In [23]:
curl -L http://localhost:80/pattern

location 1 - I use /pattern


The next cell shows what happens when you access `/pattern/wow` (we use an additional path segment).

In [24]:
curl -L http://localhost:80/pattern/wow

location 2 - I use /pattern/


## Proxy pass

Find out more in:

- The special [section of the official documentation](https://nginx.org/en/docs/http/ngx_http_proxy_module.html#proxy_pass).
- [Specific page](location/proxy_pass.ipynb) on this site.

`proxy_path` in nginx specify the URL of the proxied service. This would be a URL that nginx will request.

---

The following example defines `location /recsys` and ties to it `http://client_container/anything/config/`. 

In [25]:
cat << EOF > location_files/nginx.conf
events {}
http {
    server {
        listen 80;
        location /recsys {
            proxy_pass "http://client_container/anything/config/";
        }
    }
}
EOF

docker exec -it experiment_nginx nginx -s reload

2024/09/05 09:14:13 [notice] 37#37: signal process started


The following cell demonstrates a request to the `<nginx address>/recsys/...`, which uses httpbin to display the result of the request.

In [26]:
curl -L http://localhost:80/recsys/101

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Connection": "close", 
    "Host": "client_container", 
    "User-Agent": "curl/7.81.0"
  }, 
  "json": null, 
  "method": "GET", 
  "origin": "172.19.0.3", 
  "url": "http://client_container/anything/config//101"
}


In the `url` field you can check the exact url that nginx throws to the httpbin.

## Headers to server (`proxy_set_header`)

Allows redefining or appending fields to the request header passed to the proxied server. You can add aditional headers to the http request that will be sent to the destination server. So by using syntax `proxy_set_header <header field> <value>;`.

---

The following example changes the nginx config to add two new fields to the http header `Name` and `SecondName` and reloads nginx.

In [2]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' << EOF
events {}
http {
    server {
        listen 80;
        location / {
            proxy_pass "http://client_container/headers";
            proxy_set_header Name Fedor;
            proxy_set_header SecondName Kobak;
        }
    }
}
EOF

docker exec -it experiment_nginx nginx -s reload

2024/07/23 11:24:34 [notice] 68#68: signal process started


First, consider what happens if we just request httpbin directly.

In [3]:
curl http://localhost:81/headers

{
  "headers": {
    "Accept": "*/*", 
    "Host": "localhost:81", 
    "User-Agent": "curl/7.81.0"
  }
}


There are no additional headers - just the very basic ones generated by `curl`.

In [4]:
curl http://localhost:80

{
  "headers": {
    "Accept": "*/*", 
    "Connection": "close", 
    "Host": "client_container", 
    "Name": "Fedor", 
    "Secondname": "Kobak", 
    "User-Agent": "curl/7.81.0"
  }
}


If you compare the output of the original and proxied requests, you can see that the proxied requests have additional headers - just as we specified in the nginx configuration.